# Pandas Tutorial - Part 36

This notebook covers:
- More on NA values and NumPy differences
- Thread-safety and byte-ordering issues
- Pandas cookbook examples for time series operations

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

## More on NA Values and NumPy Differences

Continuing from Part 35, let's explore more about NA values in pandas and how pandas differs from NumPy.

### Why Not Make NumPy Like R?

Many people have suggested that NumPy should emulate the NA support present in R. However, there are significant differences in the type hierarchies between NumPy and R:

| Typeclass (NumPy) | Dtypes |
|-------------------|--------|
| numpy.floating | float16, float32, float64, float128 |
| numpy.integer | int8, int16, int32, int64 |
| numpy.unsignedinteger | uint8, uint16, uint32, uint64 |
| numpy.object_ | object_ |
| numpy.bool_ | bool_ |
| numpy.character | string_, unicode_ |

The R language, by contrast, only has a handful of built-in data types: integer, numeric (floating-point), character, and boolean. R implements NA types by reserving special bit patterns for each type to be used as the missing value.

Pandas has chosen a more Pythonic approach by using NaN for floating-point and object arrays to denote NA, and promoting integer arrays to floating when NAs must be introduced.

### Differences with NumPy

For Series and DataFrame objects, `var()` normalizes by N-1 to produce unbiased estimates of the sample variance, while NumPy's `var` normalizes by N, which measures the variance of the sample. Note that `cov()` normalizes by N-1 in both pandas and NumPy.

In [ ]:
# Create a sample Series
s = pd.Series([1, 2, 3, 4, 5])

# Compare variance calculation in pandas vs NumPy
print(f"Pandas var (normalized by N-1): {s.var()}")
print(f"NumPy var (normalized by N): {np.var(s)}")
print(f"NumPy var with ddof=1 (normalized by N-1): {np.var(s, ddof=1)}")

## Thread-Safety

As of pandas 1.0.5, pandas is not 100% thread safe. The known issues relate to the `copy()` method. If you are doing a lot of copying of DataFrame objects shared among threads, it's recommended to hold locks inside the threads where the data copying occurs.

## Byte-Ordering Issues

Occasionally you may have to deal with data that were created on a machine with a different byte order than the one on which you are running Python. A common symptom of this issue is an error like:

```
ValueError: Big-endian buffer not supported on little-endian compiler
```

To deal with this issue, you should convert the underlying NumPy array to the native system byte order before passing it to Series or DataFrame constructors.

In [ ]:
# Create a big-endian array
x = np.array(list(range(10)), '>i4')  # big endian

# Convert to native byte order
newx = x.byteswap().newbyteorder()  # force native byteorder

# Now we can create a Series
s = pd.Series(newx)
s

## Pandas Cookbook: Time Series Operations

The pandas cookbook provides many useful examples for working with time series data. Let's explore some of these examples.

### Rolling Apply with a DataFrame Returning a Scalar

Let's implement a Volume Weighted Average Price (VWAP) calculation using rolling apply on multiple columns.

In [ ]:
# Create a sample DataFrame with price and volume data
rng = pd.date_range(start='2014-01-01', periods=100)
df = pd.DataFrame({
    'Open': np.random.randn(len(rng)),
    'Close': np.random.randn(len(rng)),
    'Volume': np.random.randint(100, 2000, len(rng))
}, index=rng)

df.head()

In [ ]:
# Define VWAP function
def vwap(bars):
    return ((bars.Close * bars.Volume).sum() / bars.Volume.sum())

# Set window size
window = 5

# Calculate VWAP for each window
s = pd.concat([
    pd.Series(
        vwap(df.iloc[i:i + window]),
        index=[df.index[i + window]]
    )
    for i in range(len(df) - window)
])

# Display the results
s.round(2).head(10)

### Working with DatetimeIndex

Let's explore some useful operations with DatetimeIndex.

#### Calculate the First Day of the Month for Each Entry in a DatetimeIndex

In [ ]:
# Create a sample DatetimeIndex
dates = pd.date_range('2000-01-01', periods=5)
dates

In [ ]:
# Convert to period with month frequency, then back to timestamp
# This gives the first day of the month for each date
dates.to_period(freq='M').to_timestamp()

### Resampling Time Series Data

Resampling is a common operation with time series data. Let's explore some examples.

In [ ]:
# Create a sample time series
ts = pd.Series(np.random.randn(1000), 
               index=pd.date_range('1/1/2000', periods=1000))

# Resample to monthly frequency
monthly = ts.resample('M').mean()
monthly.head()

In [ ]:
# Resample to business month end frequency
business_monthly = ts.resample('BM').mean()
business_monthly.head()

In [ ]:
# Plot the resampled time series
fig, ax = plt.subplots(figsize=(10, 6))
monthly.plot(ax=ax, label='Monthly')
business_monthly.plot(ax=ax, label='Business Monthly')
ax.legend()
plt.title('Resampled Time Series')
plt.show()

### Merging Time Series Data

Let's explore how to merge time series data in pandas.

In [ ]:
# Create two DataFrames with overlapping index
rng = pd.date_range('2000-01-01', periods=6)
df1 = pd.DataFrame(np.random.randn(6, 3), index=rng, columns=['A', 'B', 'C'])
df2 = df1.copy()

# Display the DataFrames
print("DataFrame 1:")
print(df1)
print("\nDataFrame 2:")
print(df2)

In [ ]:
# Append the DataFrames
appended = df1.append(df2)
appended

In [ ]:
# Check for duplicate index values
appended.index.duplicated().any()

In [ ]:
# Reset the index to avoid duplicates
appended_reset = appended.reset_index()
appended_reset.head()

## Conclusion

In this notebook, we've explored:

1. More details about NA values in pandas and differences with NumPy, including:
   - Why pandas chose a different approach than R for handling NA values
   - Differences in variance calculation between pandas and NumPy

2. Thread-safety considerations in pandas

3. How to handle byte-ordering issues when working with data from different systems

4. Practical examples from the pandas cookbook for time series operations, including:
   - Rolling apply with a DataFrame returning a scalar (VWAP calculation)
   - Working with DatetimeIndex to find the first day of the month
   - Resampling time series data to different frequencies
   - Merging time series data with overlapping indices

These concepts and examples provide valuable tools for working with time series data in pandas.